# Machine Learning

Este notebook apresenta o pipeline completo de *Machine Learning* para previsão do nível do rio a partir de dados hidrológicos e meteorológicos. O objetivo é construir, treinar e avaliar modelos capazes de prever eventos críticos, como enchentes, utilizando séries temporais.

> **Resumo das etapas:**
> - Pré-processamento dos dados
> - Criação de conjuntos de treino, validação e teste
> - Definição e treinamento de modelos (MLP, LSTM, GRU)
> - Otimização de hiperparâmetros
> - Avaliação e visualização dos resultados


## 1. Initial Setup

Esta seção realiza a configuração inicial do ambiente para o experimento de *Machine Learning*.

- **Definição de variáveis e hiperparâmetros:** Estabelece limites e parâmetros para o treinamento dos modelos, como número máximo de épocas e configurações do Optuna.
- **Importação de bibliotecas:** Carrega todas as dependências necessárias para manipulação de dados, visualização, modelagem e otimização.
- **Carregamento do dataset:** Importa os dados já pré-processados, prontos para uso nos modelos.

> *Certifique-se de que todas as bibliotecas estejam instaladas e que o dataset esteja disponível no caminho especificado.*


In [ ]:
# Max allowable epochs for each model type
MAX_EPOCHS = 2000

# Hyperparameters for Hyperparameter Optimization using Optuna
OPTUNA_N_JOBS = 8  # Number of parallel jobs for hyperparameter optimization - controls how many trials run simultaneously
OPTUNA_N_TRIALS = 40  # Total number of optimization trials to run - more trials generally lead to better hyperparameter discovery
OPTUNA_PRUNE_N_STARTUP_TRIALS = 8  # Number of random trials before pruning starts - ensures diverse exploration before early stopping
OPTUNA_PRUNE_WARMUP_STEPS = 8  # Number of steps to wait before pruning can occur - prevents premature termination of promising trials

In [ ]:
PROCESSED_PATH = "../data/ANA HIDROWEB/RIO MEIA PONTE/processed.csv"

In [ ]:
# Python native libraries
from typing import List, Dict, Any, Tuple
import warnings
import uuid
import random

# Data manipulation and preprocessing
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# Visualization libraries
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# PyTorch libraries and derivatives
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import lightning as L
from lightning.pytorch import Trainer
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping
from lightning.pytorch.loggers import TensorBoardLogger
from torchinfo import summary

# Optuna Libraries
import optuna
from optuna.integration import PyTorchLightningPruningCallback

# Add the parent directory to the system path to import custom modules
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
db_path = os.path.abspath(os.path.join(os.getcwd(), "..", "db"))
if db_path not in sys.path:
    sys.path.append(db_path)

# Set precision for matrix multiplication in order to optimize performance
torch.set_float32_matmul_precision("high")

# Set random seed for reproducibility
L.seed_everything(42)

In [ ]:
from data_process import load_and_process_data_from_db

df = load_and_process_data_from_db(start_date="2014-01-01", end_date="2024-12-31")
df.index = pd.to_datetime(df.index)
df.head()

## 2. Data Preprocessing

Esta etapa prepara os dados para o treinamento dos modelos, garantindo qualidade e consistência.

- **Verificação de dados nulos:** Identifica e trata possíveis valores ausentes, evitando problemas durante o treinamento.
- **Remoção de colunas de vazão:** Exclui variáveis altamente correlacionadas (como `flow`), reduzindo redundância e evitando *overfitting*.
- **Normalização:** Aplica normalização para padronizar as escalas das variáveis, o que é fundamental para o bom desempenho de modelos baseados em gradiente.

> *O pré-processamento é essencial para garantir que os modelos aprendam padrões reais e não ruídos ou vieses dos dados.*


In [ ]:
# Count missing values in each column
def print_missing_values(data):
    missing_values = data.isnull().sum()
    print("Missing values in each column:")
    print(missing_values[missing_values > 0])


print_missing_values(df)

In [ ]:
# Remove columns containing 'flow' from the dataframe
df = df.loc[:, ~df.columns.str.contains("flow")]
df.head()

In [ ]:
scaler = StandardScaler()
# Normalize all columns except 'date_sin' and 'date_cos' (if present)
cols_to_normalize = [col for col in df.columns if col not in ["date_sin", "date_cos"]]
df_normalized = df.copy()
df_normalized[cols_to_normalize] = scaler.fit_transform(df[cols_to_normalize])
df_normalized.head()

## 3. Criação dos Datasets

Aqui são definidos os conjuntos de dados para treino, validação e teste, utilizando uma abordagem de *chunking* aleatório.

> *Essa estratégia melhora a generalização dos modelos e simula cenários reais de produção, onde padrões podem variar ao longo do tempo.*

**Vantagens do chunking:**
- Diversifica os contextos temporais em cada split
- Reduz o risco de especialização em um único regime temporal
- Mantém a dependência temporal essencial para séries temporais


**Sobre a divisão dos dados:**

Neste projeto, utilizamos a divisão dos dados por *chunking* aleatório com split sequencial dentro de cada chunk, em vez do split sequencial tradicional. Veja abaixo a diferença entre as abordagens:

- **Chunking aleatório + split sequencial (utilizado aqui):**
  - Os dados são divididos em blocos (chunks) de tamanho aleatório.
  - Cada chunk é separado sequencialmente em treino, validação e teste.
  - Isso garante que cada split contenha diferentes padrões temporais, regimes e sazonalidades.

- **Split sequencial tradicional:**
  - Os dados são divididos em três blocos contínuos: início para treino, meio para validação e fim para teste.
  - Eventos raros ou mudanças de regime podem ficar concentrados em apenas um split, dificultando a generalização.

> **Neste projeto, utilizamos o método de chunking aleatório, pois ele proporciona maior diversidade e robustez na avaliação dos modelos.**

**Vantagens de cada abordagem:**

| Estratégia                   | Vantagens principais                                                                 |
|------------------------------|-----------------------------------------------------------------------------------|
| Chunking aleatório           | Diversidade de padrões em todos os splits, simula cenários reais, melhor generalização |
| Split sequencial tradicional | Simples de implementar, reflete cenários de previsão futura, mas pode ser enviesado  |

---

**Visualização esquemática:**

Split Sequencial Tradicional:

Treino      | Validação | Teste
------------|-----------|---------
████████████|████████|████████

Chunking Aleatório + Split Sequencial (utilizado):

| Chunk 1 |   |   | Chunk 2 |   |   | ... |
|--------------|--------------|--------------|--------------|--------------|--------------|------|
| Treino | Validação | Teste | Treino | Validação | Teste | ... |
|████████████|████████|████████|████████████|████████|████████| ... |

Legenda: Cada 3 colunas representam um chunk, e dentro de cada chunk há blocos de treino, validação e teste (Treino | Validação | Teste).

*No split tradicional, cada split é um bloco contínuo. No chunking, cada chunk contém amostras de treino, validação e teste, promovendo diversidade em todos os splits.*

> *A escolha do chunking aleatório permite que o modelo aprenda e seja avaliado em diferentes contextos temporais, tornando-o mais robusto para aplicações reais.*

In [ ]:
class TimeSeriesDataset(Dataset):
    def __init__(self, data, sequence_length=24, target_col="level_downstream_max"):
        self.data = data.reset_index(drop=True)
        self.sequence_length = sequence_length
        self.target_col = self.data.columns.get_loc(target_col)

    def __len__(self):
        return len(self.data) - self.sequence_length

    def __getitem__(self, idx):
        x = self.data.iloc[
            idx : idx + self.sequence_length
        ].values  # (sequence_length, num_features)
        y = self.data.iloc[idx + self.sequence_length, self.target_col]  # scalar value
        return torch.tensor(x, dtype=torch.float32), torch.tensor(
            y, dtype=torch.float32
        )


class NonOverlappingConcatDataset(Dataset):
    def __init__(self, datasets):
        self.datasets = datasets
        self.cumulative_lengths = []
        total = 0
        for d in datasets:
            self.cumulative_lengths.append(total)
            total += len(d)
        self.total_length = total

    def __len__(self):
        return self.total_length

    def __getitem__(self, idx):
        # Find which dataset this idx belongs to
        for i in range(len(self.datasets)):
            if idx < self.cumulative_lengths[i] + len(self.datasets[i]):
                local_idx = idx - self.cumulative_lengths[i]
                return self.datasets[i][local_idx]
        raise IndexError("Index out of range")


def split_train_validation_test(
    data: pd.DataFrame, train_size: float = 0.6, val_size: float = 0.2
):
    train_end = int(len(data) * train_size)
    val_end = int(len(data) * (train_size + val_size))

    train_data = data.iloc[:train_end]
    val_data = data.iloc[train_end:val_end]
    test_data = data.iloc[val_end:]

    return train_data, val_data, test_data


def create_datasets(
    sequence_length: int,
    min_chunk_size: int = 24,
    max_chunk_size: int = 100,
) -> Tuple[
    NonOverlappingConcatDataset,
    NonOverlappingConcatDataset,
    NonOverlappingConcatDataset,
]:
    import random

    datasets_train: List[TimeSeriesDataset] = []
    datasets_validation: List[TimeSeriesDataset] = []
    datasets_test: List[TimeSeriesDataset] = []

    data = df_normalized.reset_index(drop=True)
    n = len(data)
    idx = 0
    chunk_indices = []

    # Generate random chunk indices
    while idx < n:
        chunk_size = random.randint(min_chunk_size, max_chunk_size)
        if idx + chunk_size >= n:
            # Last chunk
            if n - idx < min_chunk_size:
                break  # Discard last chunk if too small
            chunk_indices.append((idx, n))
            break
        else:
            chunk_indices.append((idx, idx + chunk_size))
            idx += chunk_size

    for start, end in chunk_indices:
        chunk = data.iloc[start:end].reset_index(drop=True)
        if not chunk.empty:
            train_data, validation_data, test_data = split_train_validation_test(chunk)
            if (
                len(train_data) > sequence_length
                and len(validation_data) > sequence_length
                and len(test_data) > sequence_length
            ):
                datasets_train.append(
                    TimeSeriesDataset(train_data, sequence_length=sequence_length)
                )
                datasets_validation.append(
                    TimeSeriesDataset(validation_data, sequence_length=sequence_length)
                )
                datasets_test.append(
                    TimeSeriesDataset(test_data, sequence_length=sequence_length)
                )

    train_dataset = NonOverlappingConcatDataset(datasets_train)
    validation_dataset = NonOverlappingConcatDataset(datasets_validation)
    test_dataset = NonOverlappingConcatDataset(datasets_test)

    return train_dataset, validation_dataset, test_dataset


# Debugging example
# fake_train, fake_validation, fake_test = create_datasets(
#     sequence_length=2,
# )  # Example with sequence length of 2

# print(fake_train[0][1])
# fake_train[0][0].shape, fake_train[0][1].shape  # Should be (2, num_features), scalar

In [ ]:
# Test for TimeSeriesDataset

# Create a simple DataFrame with increasing integers
test_df = pd.DataFrame({"A": np.arange(10)})

# Window length = 3
ts_dataset = TimeSeriesDataset(test_df, sequence_length=3, target_col="A")

print("Testing TimeSeriesDataset:")
for i in range(len(ts_dataset)):
    x, y = ts_dataset[i]
    print(f"Index {i}: x = {x.squeeze().numpy()}, y = {y.numpy()}")

# Test for NonOverlappingConcatDataset
# Create two small TimeSeriesDatasets
df1 = pd.DataFrame({"A": np.arange(5)})
df2 = pd.DataFrame({"A": np.arange(10, 15)})

ds1 = TimeSeriesDataset(df1, sequence_length=2, target_col="A")
ds2 = TimeSeriesDataset(df2, sequence_length=2, target_col="A")

concat_ds = NonOverlappingConcatDataset([ds1, ds2])

print("\nTesting NonOverlappingConcatDataset")
print("Should not overlap and should concatenate correctly:")
for i in range(len(concat_ds)):
    x, y = concat_ds[i]
    print(f"Index {i}: x = {x.squeeze().numpy()}, y = {y.numpy()}")

## 4. Definição dos Modelos

Nesta etapa são definidos os modelos de *Machine Learning* utilizados para previsão:

- **MLP (Perceptron Multicamadas):** Modelo denso tradicional, útil como baseline para séries temporais.
- **LSTM (Long Short-Term Memory):** Rede neural recorrente especializada em capturar dependências de longo prazo em séries temporais.
- **GRU (Gated Recurrent Unit):** Variante mais simples e eficiente da LSTM, também indicada para séries temporais.

Cada modelo é implementado utilizando o framework PyTorch Lightning, facilitando o treinamento, validação e integração com callbacks e loggers.

> *A escolha de diferentes arquiteturas permite comparar desempenho e identificar a abordagem mais adequada para o problema.*


In [ ]:
class MLP(L.LightningModule):
    def __init__(
        self,
        input_size,
        hidden_layers,
        learning_rate: float = 0.0001,
    ):
        super(MLP, self).__init__()
        self.save_hyperparameters()

        # MLP para cada passo temporal
        layers = []
        in_features = input_size
        for hidden_size in hidden_layers:
            layers.append(nn.Linear(in_features, hidden_size))
            layers.append(nn.ReLU())
            in_features = hidden_size
        self.step_mlp = nn.Sequential(*layers)
        self.final_layer = nn.Linear(in_features, 1)
        self.criterion = nn.MSELoss()

    def forward(self, x):
        # x: (batch, window_length, input_size)
        # Aplica o MLP em cada passo temporal    print("x type:", type(x))
        batch, window, features = x.shape
        x = x.view(-1, features)  # (batch * window_length, input_size)
        out = self.step_mlp(x)  # (batch * window_length, hidden)
        out = out.view(batch, window, -1)  # (batch, window_length, hidden)
        out = out.mean(dim=1)  # (batch, hidden) - agregação temporal
        out = self.final_layer(out)  # (batch, input_size)
        return out.squeeze(-1)  # (batch,)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=self.hparams.learning_rate)

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log("val_loss", loss)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log("test_loss", loss)
        return loss


class LSTM(L.LightningModule):
    def __init__(
        self,
        input_size,
        hidden_size,
        num_layers,
        learning_rate: float = 0.0001,
    ):
        super(LSTM, self).__init__()
        self.save_hyperparameters()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        self.criterion = nn.MSELoss()

    def forward(self, x):
        out, _ = self.lstm(x)  # (batch, sequence_length, hidden_size)
        return self.fc(out[:, -1, :]).squeeze(-1)  # (batch,)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=self.hparams.learning_rate)

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log("val_loss", loss)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log("test_loss", loss)
        return loss


class GRU(L.LightningModule):
    def __init__(
        self,
        input_size,
        hidden_size,
        num_layers,
        learning_rate: float = 0.0001,
    ):
        super(GRU, self).__init__()
        self.save_hyperparameters()
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        self.criterion = nn.MSELoss()

    def forward(self, x):
        out, _ = self.gru(x)
        return self.fc(out[:, -1, :]).squeeze(-1)  # (batch,)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=self.hparams.learning_rate)

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log("val_loss", loss)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log("test_loss", loss)
        return loss


print(summary(MLP(input_size=5, hidden_layers=[10, 10])))
print("\n")
print(summary(LSTM(input_size=5, hidden_size=10, num_layers=2)))
print("\n")
print(summary(GRU(input_size=5, hidden_size=10, num_layers=2)))

## 5. Treinamento dos Modelos

Esta seção abrange todo o processo de treinamento, validação e otimização dos modelos.

- **Funções de treinamento:** Automatizam o processo de treino, validação e avaliação dos modelos, utilizando *callbacks* para checkpoint e early stopping.
- **Otimização de hiperparâmetros:** Utiliza o Optuna para encontrar as melhores configurações de cada modelo, acelerando o processo com paralelismo e pruning.
- **Laço de treinamento:** Executa o treinamento final dos modelos com os melhores hiperparâmetros encontrados.

> *O uso de automação e otimização garante experimentos reprodutíveis e eficientes.*


### 5.1. Funções de treinamento

As funções desta subseção são responsáveis por:

- Treinar os modelos com os dados de treino e validação
- Avaliar o desempenho nos dados de teste
- Gerenciar logs, checkpoints e callbacks

> *Utilize estas funções para garantir um fluxo de treinamento padronizado e monitorado.*


In [ ]:
def train_model(
    model,
    train_data,
    val_data,
    logger,
    epochs,
    batch_size,
    callbacks=[],
    verbose=True,
) -> None:
    """
    Treina o modelo com os dados de treino e validação.
    """

    train_loader = DataLoader(
        train_data, batch_size=batch_size, shuffle=True, drop_last=True
    )
    validation_loader = DataLoader(
        val_data, batch_size=4, shuffle=False, drop_last=True
    )

    trainer = Trainer(
        max_epochs=epochs,
        accelerator="auto",
        devices=1,
        logger=logger,
        enable_progress_bar=verbose,
        log_every_n_steps=1,
        callbacks=callbacks,
        deterministic=True,
        check_val_every_n_epoch=1,
        enable_model_summary=verbose,
    )
    trainer.fit(model, train_loader, validation_loader)


def evaluate_model(model, test_data, logger, batch_size=4):
    """
    Avalia o modelo com os dados de teste.
    """
    print("\nEvaluating model. Using the test dataset.")
    test_loader = DataLoader(
        test_data, batch_size=batch_size, shuffle=False, drop_last=True
    )

    trainer = Trainer(
        accelerator="auto",
        devices=1,
        logger=logger,
        enable_progress_bar=True,
        deterministic=True,
    )
    trainer.test(model, test_loader)


def run_experiment(
    model_class: type[L.LightningModule],
    name: str,
    model_kwargs: Dict[str, Any] = {},
    train_kwargs: Dict[str, Any] = {},
    evaluate: bool = True,
    extra_callbacks: List[L.Callback] = [],
    verbose: bool = True,
    id: str = "",
):
    """
    Executa um experimento de treinamento e avaliação do modelo.
    """
    # Logger
    logger = TensorBoardLogger(save_dir=os.getcwd(), name=f"lightning_logs/{name}")

    checkpoint_callback = ModelCheckpoint(
        monitor="val_loss", mode="min", save_top_k=1, filename=f"best_model_{id}"
    )

    early_stopping_callback = EarlyStopping(
        monitor="val_loss",
        min_delta=1e-4,
        patience=10,
        mode="min",
        strict=False,
    )

    train_dataset, validation_dataset, test_dataset = create_datasets(
        sequence_length=train_kwargs.get("sequence_length", 5)
    )

    # Remove 'sequence_length' from train_kwargs if present
    train_kwargs = dict(train_kwargs)  # make a copy to avoid side effects
    train_kwargs.pop("sequence_length", None)

    input_size = train_dataset[0][0].shape[1]  # Assuming all inputs have the same shape

    model = model_class(input_size=input_size, **(model_kwargs or {}))

    train_model(
        model,
        train_dataset,
        validation_dataset,
        logger=logger,
        **(train_kwargs or {}),
        callbacks=[checkpoint_callback, early_stopping_callback, *extra_callbacks],
        verbose=verbose,
    )

    best_validation_loss = checkpoint_callback.best_model_score
    best_model_path = checkpoint_callback.best_model_path

    # To view the logs, you can use TensorBoard.
    # If 'logs' is a SummaryWriter, get its log_dir and launch TensorBoard:
    print(f"To view logs, run in terminal:\n  tensorboard --logdir {logger.log_dir}")
    print("Best model saved at:", best_model_path)

    if evaluate:
        best_model = model_class.load_from_checkpoint(best_model_path, **model_kwargs)
        evaluate_model(best_model, test_dataset, logger, batch_size=4)

    return {
        "best_validation_loss": (
            best_validation_loss.item() if best_validation_loss is not None else None
        ),
        "best_model_path": best_model_path,
    }

### 5.2. Funções de otimização de hiperparâmetros

Esta subseção implementa a busca automática pelos melhores hiperparâmetros dos modelos utilizando o Optuna.

- **Definição do espaço de busca:** Cada modelo possui um conjunto de hiperparâmetros a serem otimizados (ex: número de camadas, taxa de aprendizado, tamanho da janela temporal).
- **Função objetivo:** Avalia o desempenho do modelo para cada combinação de hiperparâmetros.
- **Execução dos experimentos:** Roda múltiplos experimentos em paralelo, utilizando pruning para acelerar a busca.

> *A otimização de hiperparâmetros é fundamental para extrair o máximo desempenho dos modelos.*


In [ ]:
def get_optimization_args(model_class: L.LightningModule, trial):
    """
    Retorna os argumentos necessários para executar o experimento.
    """
    if model_class == MLP:
        model_kwargs = {
            "hidden_layers": trial.suggest_categorical(
                "hidden_layers",
                (
                    [32, 16],
                    [48, 24],
                    [64, 32],
                    [96, 48],
                    [128, 64],
                    [64, 32, 16],
                    [128, 64, 32],
                ),
            ),
            "learning_rate": trial.suggest_float("learning_rate", 1e-5, 1e-3, log=True),
        }
        train_kwargs = {
            "epochs": MAX_EPOCHS,
            "batch_size": trial.suggest_int("batch_size", 2, 32),
            "sequence_length": trial.suggest_int("sequence_length", 2, 15),
        }
    elif model_class == LSTM:
        model_kwargs = {
            "hidden_size": trial.suggest_int("hidden_size", 16, 128),
            "num_layers": trial.suggest_int("num_layers", 1, 3),
            "learning_rate": trial.suggest_float("learning_rate", 1e-7, 1e-3, log=True),
        }
        train_kwargs = {
            "epochs": MAX_EPOCHS,
            "batch_size": trial.suggest_int("batch_size", 2, 32),
            "sequence_length": trial.suggest_int("sequence_length", 2, 15),
        }
    elif model_class == GRU:
        model_kwargs = {
            "hidden_size": trial.suggest_int("hidden_size", 16, 128),
            "num_layers": trial.suggest_int("num_layers", 1, 3),
            "learning_rate": trial.suggest_float("learning_rate", 1e-7, 1e-3, log=True),
        }
        train_kwargs = {
            "epochs": MAX_EPOCHS,
            "batch_size": trial.suggest_int("batch_size", 8, 32),
            "sequence_length": trial.suggest_int("sequence_length", 3, 12),
        }
    else:
        raise ValueError(f"Unsupported model class: {model_class}")
    return model_kwargs, train_kwargs


def get_objective_fn(model_class, name):
    """
    Retorna uma função de objetivo para otimização de hiperparâmetros.
    """

    def objective(trial: optuna.trial.Trial) -> float:
        # Ajusta os hiperparâmetros com base no trial
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=UserWarning)
            model_kwargs, train_kwargs = get_optimization_args(model_class, trial)

        optuna_pruning_callback = PyTorchLightningPruningCallback(
            trial, monitor="val_loss"
        )

        results = run_experiment(
            model_class=model_class,
            name=name,
            model_kwargs=model_kwargs,
            train_kwargs=train_kwargs,
            evaluate=False,
            extra_callbacks=[optuna_pruning_callback],
            verbose=False,
            id=uuid.uuid4().hex[:8],
        )
        return results["best_validation_loss"]

    return objective


def run_optimization(objective_fn, n_trials=OPTUNA_N_TRIALS) -> optuna.study.Study:
    pruner = optuna.pruners.MedianPruner(
        n_startup_trials=OPTUNA_PRUNE_N_STARTUP_TRIALS,
        n_warmup_steps=OPTUNA_PRUNE_WARMUP_STEPS,
    )
    study = optuna.create_study(direction="minimize", pruner=pruner)
    study.optimize(
        objective_fn, n_trials=n_trials, n_jobs=OPTUNA_N_JOBS, show_progress_bar=True
    )

    print("Best trial:")
    trial = study.best_trial
    print(f"  Value: {trial.value}")
    print("  Params: ")
    for key, value in trial.params.items():
        print(f"    {key}: {value}")

    return study

### 5.3. Laço de treinamento final

Aqui são executados os treinamentos finais dos modelos MLP, LSTM e GRU utilizando os melhores hiperparâmetros encontrados na etapa anterior.

- **Treinamento final:** Cada modelo é treinado do zero com os hiperparâmetros otimizados.
- **Avaliação:** O desempenho é avaliado nos dados de teste, permitindo comparação justa entre as arquiteturas.

> *Os resultados obtidos aqui servirão de base para análise e conclusões posteriores.*


In [ ]:
mlp_objective_fn = get_objective_fn(
    MLP,
    "mlp",
)

study_mlp = run_optimization(mlp_objective_fn, n_trials=OPTUNA_N_TRIALS)

mlp_optimized_results = run_experiment(
    MLP,
    "mlp_optimized",
    model_kwargs={
        "hidden_layers": study_mlp.best_trial.params["hidden_layers"],
        "learning_rate": study_mlp.best_trial.params["learning_rate"],
    },
    train_kwargs={
        "epochs": MAX_EPOCHS,
        "batch_size": study_mlp.best_trial.params["batch_size"],
        "sequence_length": study_mlp.best_trial.params["sequence_length"],
    },
    evaluate=True,
    verbose=True,
)

print("\nMLP Optimized Results:", mlp_optimized_results)

In [ ]:
lstm_objective_fn = get_objective_fn(
    LSTM,
    "lstm",
)
study_lstm = run_optimization(lstm_objective_fn, n_trials=OPTUNA_N_TRIALS)
lstm_optimized_results = run_experiment(
    LSTM,
    "lstm_optimized",
    model_kwargs={
        "hidden_size": study_lstm.best_trial.params["hidden_size"],
        "num_layers": study_lstm.best_trial.params["num_layers"],
        "learning_rate": study_lstm.best_trial.params["learning_rate"],
    },
    train_kwargs={
        "epochs": MAX_EPOCHS,
        "batch_size": study_lstm.best_trial.params["batch_size"],
    },
    evaluate=True,
    verbose=True,
)
print("\nLSTM Optimized Results:", lstm_optimized_results)

In [ ]:
gru_objective_fn = get_objective_fn(
    GRU,
    "gru",
)
study_gru = run_optimization(gru_objective_fn, n_trials=OPTUNA_N_TRIALS)
gru_optimized_results = run_experiment(
    GRU,
    "gru_optimized",
    model_kwargs={
        "hidden_size": study_gru.best_trial.params["hidden_size"],
        "num_layers": study_gru.best_trial.params["num_layers"],
        "learning_rate": study_gru.best_trial.params["learning_rate"],
    },
    train_kwargs={
        "epochs": MAX_EPOCHS,
        "batch_size": study_gru.best_trial.params["batch_size"],
    },
    evaluate=True,
    verbose=True,
)
print("\nGRU Optimized Results:", gru_optimized_results)

## 6. Avaliação dos Modelos

Esta etapa compara o desempenho dos modelos treinados, utilizando métricas quantitativas e visualizações gráficas.

- **Comparação visual:** Gráficos que mostram as previsões dos modelos versus os valores reais ao longo do tempo, destacando as regiões de treino, validação e teste.
- **Análise de desempenho:** Permite identificar padrões, tendências, possíveis overfits e diferenças de desempenho entre os modelos.

> *A avaliação visual é essencial para entender o comportamento dos modelos em diferentes cenários e períodos da série temporal.*


### 6.1. Visualização

Os gráficos desta subseção facilitam a análise comparativa entre as previsões dos modelos (MLP, LSTM, GRU) e os valores reais.

- **Cores de fundo:** Indicam os splits de treino, validação e teste.
- **Linhas:** Representam as previsões suavizadas de cada modelo e os valores reais.
- **Divisão em painéis:** Permite visualizar segmentos da série temporal de forma clara e organizada.

> *Com esta visualização, é possível identificar rapidamente regiões de bom ou mau desempenho, além de padrões sazonais ou eventos extremos.*

In [ ]:
# --- 1. Define split colors and model colors ---
split_colors = {
    "train": "#b39ddb",  # stronger purple
    "validation": "#ffb6b9",  # stronger pink
    "test": "#b2fab4",  # stronger green
}
model_colors = {
    "Real": "black",
    "MLP": "tab:blue",
    "LSTM": "tab:orange",
    "GRU": "tab:green",
}


# --- 2. Recreate chunking and split mapping ---
def get_chunk_splits(data, sequence_length=5, min_chunk_size=24, max_chunk_size=100):
    n = len(data)
    idx = 0
    chunk_indices = []
    while idx < n:
        chunk_size = random.randint(min_chunk_size, max_chunk_size)
        if idx + chunk_size >= n:
            if n - idx < min_chunk_size:
                break
            chunk_indices.append((idx, n))
            break
        else:
            chunk_indices.append((idx, idx + chunk_size))
            idx += chunk_size

    split_map = {}
    chunk_boundaries = []
    chunk_labels = []
    for chunk_id, (start, end) in enumerate(chunk_indices):
        chunk_len = end - start
        train_end = start + int(chunk_len * 0.6)
        val_end = start + int(chunk_len * 0.8)
        # Assign split type for each index in chunk
        for i in range(start, train_end):
            split_map[i] = ("train", chunk_id)
        for i in range(train_end, val_end):
            split_map[i] = ("validation", chunk_id)
        for i in range(val_end, end):
            split_map[i] = ("test", chunk_id)
        chunk_boundaries.append((start, end))
        chunk_labels.append(f"Chunk {chunk_id}")
    return split_map, chunk_boundaries, chunk_labels


split_map, chunk_boundaries, chunk_labels = get_chunk_splits(
    df_normalized, sequence_length=5
)

# --- 3. Prepare data for plotting ---
x_labels = df_normalized.index  # Use date or index
target_col = "level_downstream_max"
real_values = df_normalized[target_col].values

# For each sample, get split type and chunk id
split_types = []
chunk_ids = []
for i in range(len(df_normalized)):
    split, chunk_id = split_map.get(i, ("none", -1))
    split_types.append(split)
    chunk_ids.append(chunk_id)


# --- 4. Prediction function for a model ---
def get_model_predictions(model, data, sequence_length=5):
    model.eval()
    preds = [None] * len(data)
    with torch.no_grad():
        for i in range(len(data) - sequence_length):
            x = data.iloc[i : i + sequence_length].values
            x_tensor = (
                torch.tensor(x, dtype=torch.float32).unsqueeze(0).to(device)
            )  # (1, seq, features)
            y_pred = model(x_tensor).cpu().numpy().squeeze()
            preds[i + sequence_length] = y_pred
    return preds


# Set the paths for the best models
mlp_checkpoint_path = mlp_optimized_results["best_model_path"]
lstm_checkpoint_path = lstm_optimized_results["best_model_path"]
gru_checkpoint_path = gru_optimized_results["best_model_path"]

device = "cuda" if torch.cuda.is_available() else "cpu"

# Load best models (replace with your checkpoint paths)
mlp_model = MLP.load_from_checkpoint(mlp_checkpoint_path).to(device)
lstm_model = LSTM.load_from_checkpoint(lstm_checkpoint_path).to(device)
gru_model = GRU.load_from_checkpoint(gru_checkpoint_path).to(device)

mlp_preds = get_model_predictions(
    mlp_model,
    df_normalized,
    sequence_length=study_mlp.best_trial.params["sequence_length"],
)
lstm_preds = get_model_predictions(
    lstm_model,
    df_normalized,
    sequence_length=study_lstm.best_trial.params["sequence_length"],
)
gru_preds = get_model_predictions(
    gru_model,
    df_normalized,
    sequence_length=study_gru.best_trial.params["sequence_length"],
)

# --- 5. Plotting with smoothing (split into 4 rows) ---
n_splits = 8
n_samples = len(real_values)
split_size = n_samples // n_splits

fig, axes = plt.subplots(
    nrows=n_splits, ncols=1, figsize=(18, 6 * n_splits), sharey=True
)

window = 7  # Smoothing window size (days/samples)

for split_idx in range(n_splits):
    ax = axes[split_idx]
    start_idx = split_idx * split_size
    end_idx = (split_idx + 1) * split_size if split_idx < n_splits - 1 else n_samples

    # Get the date indices for this split
    split_dates = x_labels[start_idx:end_idx]

    # For each sample in this split, shade background according to split type
    last_split = None
    region_start = 0
    for i in range(start_idx, end_idx):
        split = split_types[i]
        if split != last_split or i == end_idx - 1:
            if last_split is not None:
                region_end = i if split != last_split else i + 1
                color = split_colors.get(last_split, "#f0f0f0")
                # Use date indices for axvspan
                ax.axvspan(
                    split_dates[region_start - start_idx],
                    (
                        split_dates[region_end - start_idx - 1]
                        if region_end - start_idx - 1 < len(split_dates)
                        else split_dates[-1]
                    ),
                    color=color,
                    alpha=0.25,
                    zorder=0,
                )
            region_start = i
            last_split = split

    # Smooth real values for this split
    real_smooth = (
        pd.Series(real_values[start_idx:end_idx], index=split_dates)
        .rolling(window, min_periods=1)
        .mean()
    )
    ax.plot(
        split_dates,
        real_smooth,
        label="Real (smoothed)" if split_idx == 0 else None,
        color=model_colors["Real"],
        marker=".",
        markersize=5,
        linewidth=1.5,
        zorder=2,
    )

    # Plot smoothed model predictions (skip None values)
    for name, preds, color in [
        ("MLP", mlp_preds, model_colors["MLP"]),
        ("LSTM", lstm_preds, model_colors["LSTM"]),
        ("GRU", gru_preds, model_colors["GRU"]),
    ]:
        idxs = [i for i in range(start_idx, end_idx) if preds[i] is not None]
        vals = [preds[i] for i in idxs]
        if vals:
            idx_dates = [x_labels[i] for i in idxs]
            vals_smooth = (
                pd.Series(vals, index=idx_dates).rolling(window, min_periods=1).mean()
            )
            ax.plot(
                idx_dates,
                vals_smooth,
                label=f"{name} (smoothed)" if split_idx == 0 else None,
                color=color,
                marker=".",
                markersize=5,
                linewidth=1.5,
                alpha=0.8,
                zorder=3,
            )
    # X-axis ticks at chunk starts within this split (now using dates)
    split_tick_indices = []
    split_tick_labels = []
    for chunk_start, chunk_end in chunk_boundaries:
        # Only consider chunks that overlap with this split
        if chunk_end <= start_idx or chunk_start >= end_idx:
            continue
        chunk_len = chunk_end - chunk_start
        train_start = chunk_start
        val_start = chunk_start + int(chunk_len * 0.6)
        test_start = chunk_start + int(chunk_len * 0.8)
        for idx in [train_start, val_start, test_start]:
            if start_idx <= idx < end_idx:
                split_tick_indices.append(idx)
                split_tick_labels.append(str(x_labels[idx].date()))

    ax.set_xticks([x_labels[i] for i in split_tick_indices])
    ax.set_xticklabels(split_tick_labels, rotation=75)
    ax.set_xlabel(f"Date (Split {split_idx+1})")
    if split_idx == 0:
        ax.set_ylabel(target_col)
    ax.set_title(
        f"Real vs. Model Predictions (Smoothed) - Split {split_idx+1}/{n_splits}"
    )

# Legends for splits (background) and models (lines) only on the first subplot
split_patches = [
    mpatches.Patch(color=split_colors[k], label=k.capitalize()) for k in split_colors
]
model_lines = [
    plt.Line2D([0], [0], color=model_colors[k], marker=".", label=f"{k} (smoothed)")
    for k in model_colors
]
axes[0].legend(handles=split_patches + model_lines, loc="upper right", ncol=2)

plt.tight_layout()
plt.show()

## 7. Conclusões

*Espaço reservado para as conclusões finais após análise dos resultados dos modelos.*

> **Inclua aqui as principais descobertas, limitações e sugestões para trabalhos futuros.**


*Espaço reservado para métricas quantitativas (RMSE, MAE, etc.) e observações finais após o treinamento dos modelos.*
